### Importing the libraries

TO DO:
- Add Token Embedding Layer
- Add Positional Embedding Layer
- Add Linear Layer

In [1]:
import torch

### Defining the Hyperparameters

In [2]:
batch_size = 32 # how many independent sequences to process in parallel
block_size = 8 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

### Loading the Dataset

In [3]:
# Load the tiny shakespeare dataset
dataset = "tiny_shakespeare.txt"

# Load the dataset into a string
with open(dataset, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

### Defining the Encoder and Decoder

In [4]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

### Splitting the Data

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

### Data Loading

In [6]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of input-target pairs
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Loss Function

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Single Head

In [8]:
class Head(torch.nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = torch.nn.Linear(n_embed, head_size, bias=False)
        self.query = torch.nn.Linear(n_embed, head_size, bias=False)
        self.value = torch.nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.nn.functional.softmax(wei, dim=-1)
        out = wei @ v
        return out

### Multi Head Attention

In [9]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(n_embed, n_embed)
    
    def forward(self, x):
        return self.proj(torch.cat([h(x) for h in self.heads], dim=-1))

### Feed Forward Network

In [10]:
class FeedForward(torch.nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(n_embed, n_embed * 4),
            torch.nn.ReLU(),
            torch.nn.Linear(n_embed * 4, n_embed)
        )
    
    def forward(self, x):
        return self.net(x)

### Layer Norm

In [11]:
class LayerNorm:

    def __init__(self, dim, eps = 1e-5, momentum = 0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        x_mean = x.mean(1, keepdim=True)
        x_var = x.var(1, keepdim=True)
        x_hat = (x - x_mean) / torch.sqrt(x_var + self.eps)
        return self.gamma * x_hat + self.beta

### Complete Decoder Block

In [12]:
class Block(torch.nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = torch.nn.LayerNorm(n_embed)
        self.ln2 = torch.nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual connection
        x = x + self.ffwd(self.ln2(x)) # residual connection
        return x

### Bigram Model

In [13]:
torch.manual_seed(1337)

class BigramLanguageModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embed)
        self.block = Block(n_embed, n_head = 4)
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def forward(self, idx, y = None):
        # idx is (B, T) tensor of indices.
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.block(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        # Loss
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = torch.functional.F.cross_entropy(logits, y)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = torch.nn.functional.softmax(logits, dim = -1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx

In [14]:
model = BigramLanguageModel()
model = model.to(device)

### Training the Model

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [16]:
for i in range(max_iters):
    
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    # perform backpropagation
    loss.backward()

    # update the weights
    optimizer.step()

    # zero the gradients
    optimizer.zero_grad(set_to_none = True)

step 0: train loss 4.4829, val loss 4.4847
step 300: train loss 2.5911, val loss 2.6113
step 600: train loss 2.4491, val loss 2.4611
step 900: train loss 2.3795, val loss 2.3911
step 1200: train loss 2.3252, val loss 2.3444
step 1500: train loss 2.2872, val loss 2.3080
step 1800: train loss 2.2643, val loss 2.2824
step 2100: train loss 2.2370, val loss 2.2516
step 2400: train loss 2.2135, val loss 2.2453
step 2700: train loss 2.2069, val loss 2.2390
step 3000: train loss 2.1922, val loss 2.2167
step 3300: train loss 2.1854, val loss 2.2059
step 3600: train loss 2.1651, val loss 2.1953
step 3900: train loss 2.1544, val loss 2.2013
step 4200: train loss 2.1530, val loss 2.2037
step 4500: train loss 2.1539, val loss 2.1900
step 4800: train loss 2.1206, val loss 2.1901
step 5100: train loss 2.1241, val loss 2.1641
step 5400: train loss 2.0929, val loss 2.1721
step 5700: train loss 2.1035, val loss 2.1505
step 6000: train loss 2.0970, val loss 2.1531
step 6300: train loss 2.0912, val loss 2

In [18]:
context = torch.zeros((1, 1), dtype = torch.long) # Since idx 0 is a new line character
out = model.generate(context, max_new_tokens = 200)
print(decode(out[0].tolist()))


And in slaw.

MEO:
Nak oukess, and my werigh the weemble:
Do lowTle and sus but with ange in thun and by I kinge.
Then:
Wey, the upust he baring:
Orred,
An poof I shom thou dresself, good, garef; ourp
